# Image testing for pre-processing

## Importing Modules

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import csv
import json
from os import listdir
from os.path import isfile, join

from ManageLeafData import ManageLeafData as mld

## Loading image

In [2]:
# Global path must not be changed unless machine has been changed.
global_path = 'Collected Leaf data/Our Collection/Natural/'
# Local path can be changed anywhere, so before using it, specify the required path.
local_path = ''

In [3]:
img_1 = cv.imread(global_path + 'IMG_20230922_171310509.jpg')
cv.imshow('Raw Image', img_1)
cv.waitKey(0)
cv.destroyAllWindows()

In [4]:
image_lst = ['IMG_20230922_171255218.jpg', 'IMG20230922165200.jpg', 'IMG20230922170108.jpg', 'IMG20230922170842.jpg', 'IMG20231004155721.jpg']
# In this list the last image is from Botany collection, thus its path must be change.

## Image Smoothening

In [5]:
def GaussianSmoothing(mat):
    _pi = 3.14
    _sd = np.std(mat)
    _e = 2.718
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            try:
                mat[y][x] = ((1 / (2 * _pi * (_sd ** 2)))) * (_e ** (((x ** 2) + (y ** 2)) * (1 / (2 * (x ** 2)))))
            except ZeroDivisionError:
                pass
    return mat

In [6]:
# img_1_myGblur = GaussianSmoothing(cv.cvtColor(img_1, cv.COLOR_BGR2GRAY))

# img_1_myGblur_resized = cv.resize(img_1_myGblur, (1000, 1000))
# cv.imshow('Image Gaussian Blur & Resized', img_1_myGblur_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [7]:
# img_1_resized_Gblur = cv.GaussianBlur(img_1_resized, (7,7), 0)
# cv.imshow('Image Resized & Gaussian Blur', img_1_resized_Gblur)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur = cv.GaussianBlur(img_1, (7,7), 0)
cv.imshow('Image Gaussian Blur', img_1_Gblur)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Resizing

In [8]:
# img_1_resized = cv.resize(img_1, (int(img_1.shape[1] / 6), int(img_1.shape[0] / 6)))
# cv.imshow('Image Resized', img_1_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur_resized = cv.resize(img_1_Gblur, (1000, 1000))
cv.imshow('Image Gaussian Blur & Resized', img_1_Gblur_resized)
cv.waitKey(0)
cv.destroyAllWindows()

## Gray Scaling

In [9]:
img_1_Gblur_resized_gray = cv.cvtColor(img_1_Gblur_resized, cv.COLOR_BGR2GRAY)
cv.imshow('Image Gaussian Blur & Resized & Gray', img_1_Gblur_resized_gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
# Function to map each intensity level to output intensity level. 
def pixelVal(pix, r1, s1, r2, s2): 
    if (0 <= pix and pix <= r1): 
        return (s1 / r1)*pix 
    elif (r1 < pix and pix <= r2): 
        return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1 
    else: 
        return ((255 - s2)/(255 - r2)) * (pix - r2) + s2 

# Define parameters. 
r1 = 110 # controls darkness
s1 = 0.15 # lower bound
r2 = 7 # controls lightness
s2 = 250 # upper bound

# Vectorize the function to apply it to each value in the Numpy array. 
pixelVal_vec = np.vectorize(pixelVal) 

# Apply contrast stretching. 
img_1_contrast_stretched = pixelVal_vec(img_1_Gblur_resized_gray, r1, s1, r2, s2) 

cv.imshow('Contrast Stretching', img_1_contrast_stretched)
cv.waitKey(0)
cv.destroyAllWindows()


In [11]:
img_thresh = cv.threshold(img_1_Gblur_resized_gray, 127, 255, cv.THRESH_BINARY)
cv.imshow('Thresholding', img_thresh[1])
cv.waitKey(0)
cv.destroyAllWindows()

# print(img_thresh)

In [12]:
img_adptthresh = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 0)
cv.imshow('Adaptive Thresholding', img_adptthresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
img_adptthresh1 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 3.5)
cv.imshow('Adaptive Thresholding 1', img_adptthresh1)
# img_adptthresh2 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 4)
# cv.imshow('Adaptive Thresholding 2', img_adptthresh2)
img_adptthresh3 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 1001, 3.5)
cv.imshow('Adaptive Thresholding 3', img_adptthresh3)
cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
img_hist = cv.equalizeHist(img_thresh[1])
cv.imshow('Histogram equlization', img_hist)
cv.waitKey(0)
cv.destroyAllWindows()

## Combining these processes in a single function

In [15]:
def ImagePreprocess_Combination_1(name: str):
    cv.imshow('Final Image', pixelVal_vec(cv.resize(cv.GaussianBlur(cv.imread(name)
                                           , (7,7), 0), 
                           (1000, 1000)), 
                r1, s1, r2, s2))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [16]:
def ImagePreprocess_Combination_2(name: str):
    cv.imshow('Final Image', cv.equalizeHist(cv.adaptiveThreshold(cv.cvtColor(cv.resize(cv.GaussianBlur(cv.imread(name)
                                                                                                        , (7,7), 0)
                                                                                        , (1000, 1000))
                                                                              , cv.COLOR_BGR2GRAY)
                                                                  , 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 5)
                                            )
             )
    cv.waitKey(0)
    cv.destroyAllWindows()

In [17]:
for image in image_lst:
    ImagePreprocess_Combination_2(global_path + image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'


In [18]:
import pandas as pd

df = pd.read_csv('Leaf_Data_Mapped.csv')
df

,Image,Common name,Scientific name,Class number
0,IMG20231004154634.jpg,Peepla,Ficus religiosa,0
1,IMG20231004154651.jpg,Peepla,Ficus religiosa,0
2,IMG20231004154748.jpg,Peepla,Ficus religiosa,0
3,IMG20231004154809.jpg,Peepla,Ficus religiosa,0
4,IMG20231004154917.jpg,Peepla,Ficus religiosa,0
...,...,...,...,...
92,IMG20231004172618.jpg,Pipal,Ficus amplissima,19
93,IMG20231004172657.jpg,Pipal,Ficus amplissima,19
94,IMG20231004172723.jpg,Pipal,Ficus amplissima,19
95,IMG20231004172803.jpg,Pipal,Ficus amplissima,19


In [19]:
local_path = 'Collected Leaf data/Botany Collection/Natural/'
_cnt = 0
for image in df['Image']:
    ImagePreprocess_Combination_2(local_path + image)
    if _cnt == 10: break
    _cnt += 1

In [20]:
image_blackBG_lst = mld.getImageNameList('BlackBG/')
image_blackBG_lst

['IMG_20231004_161936.jpg',
 'IMG_20231004_162013.jpg',
 'IMG_20231004_162043.jpg',
 'IMG_20231004_164637.jpg',
 'IMG_20231004_171344.jpg',
 'IMG_20231004_171917.jpg',
 'IMG_20231004_173050.jpg',
 'IMG_20231004_203638.jpg',
 'IMG_20231004_211127.jpg',
 'IMG_20231004_211747.jpg']

In [21]:
local_path = 'BlackBG/'
for image in image_blackBG_lst:
    ImagePreprocess_Combination_2(local_path + image)

# NEXT TASK

1) Train a model on the black background images and also pre-process them along with segmentation.
2) Create a prog that takes natural images, then creates multiple combinations with varying parameters of thresholding func, then puts the processed image into the model and gets the accuracy for any single class.
3) Now train that model with that new image.

=============================================================================================================================

# EXPERIMENTS

### New Species
#####  (sp: signature plant) (g: Genus)

1) Ivora Coccinea
2) Bougainvillea glabra
3) karmda
4) Tacoma stans
5) (Genus) wedelia,(sp) trilobata
6) annona squamasa
7) lantana camera
8) aegle marmelos
9) pedilanthus tithumalodes
10) vitex negundo
11) dracaena reflexa
12) ipomea carica
13) cocculus hirsutus
14) dracaena angustifolia
15) nerium oleander
16) aehyranthes aspera
17) abution indieron
18) aristolochia [htoralis] pending_
19) tetraploid mulberry
20) euphorbia tithymaloides
21) acalypha wilkesiana
22) Murraya paniculata
23) pelargonium sp.
24) lawsonia lnermis
25) (genus) euphorbia, (sp) milii
26) polyseias scutellaria
27) justicia adhatoda
28) limonia acidissima (wood apple)
29) dracena reflexa  <--- This is repeated in the new species itself
30) polyscias fruticoss
31) kalanchoe brasilliensis
32) thunbergia fragrenf
33) jasminum sambac
34) Madagascar rubba vine cryptosteria madagasriensis
35) acter psudocreth
36) myoporum Laetum (sp: signature plant)
37) Epipremnun aureum 

## Rectifying if any species are redundant. And creating a new json file New_Species.json

In [27]:
# Function to process the new species and match them with old species
def process_new_species(old_species, new_species):
    new_species_list = new_species.split('\n')
    for word in new_species_list:
        for key, value in old_species.items():
            if word in value:
                new_species_list.remove(word)
                break
    return new_species_list

# Function to append new species to old species
def append_new_species(old_species, new_species_list):
    new_key = len(old_species)
    for species in new_species_list:
        old_species[str(new_key)] = ['', species]
        new_key += 1

# Read old species from Species.json
old_species = mld.readSpeciesJsonFile()

# New species string
new_species_string = """Ixora coccinea
Bougainvillea glabra
karmda
Tacoma stans
(Genus) wedelia,(sp) trilobata
annona squamasa
lantana camera
aegle marmelos
pedilanthus tithumalodes
vitex negundo
dracaena reflexa
ipomea carica
cocculus hirsutus
dracaena angustifolia
nerium oleander
aehyranthes aspera
abution indieron
aristolochia [htoralis] pending_
tetraploid mulberry
euphorbia tithymaloides
acalypha wilkesiana
Murraya paniculata
pelargonium sp.
lawsonia lnermis
(genus) euphorbia, (sp) milii
polyseias scutellaria
justicia adhatoda
limonia acidissima (wood apple)
polyscias fruticoss
kalanchoe brasilliensis
thunbergia fragrenf
jasminum sambac
Madagascar rubba vine cryptosteria madagasriensis
acter psudocreth
myoporum Laetum (sp: signature plant)
Epipremnun aureum"""

# Process new species
new_species_list = process_new_species(old_species, new_species_string)

# Append new species to old species
append_new_species(old_species, new_species_list)

# Write combined species to New_Species.json
with open("New_Species.json", 'w') as file:
    json.dump(old_species, file, indent=4)


### Transferring Species from New_Species_with_CommonNames.json to Species.json

In [3]:
def capitalize_all_words(string):
    return string.title()

with open('New_Species_with_CommonNames.json', 'r') as file:
    new_data = dict(json.load(file))
    for key, value in new_data.items():
        value[0], value[1] = capitalize_all_words(value[0]), capitalize_all_words(value[1])
        mld.addNewSpecies(value)

### Testing to get all images of each species from csv. One image of each species.

In [6]:
csv_data = pd.read_csv('Leaf_Data_Mapped.csv')
csv_data

,Image,Common name,Scientific name,Class number
0,IMG20231004154634.jpg,Peepla,Ficus religiosa,0
1,IMG20231004154651.jpg,Peepla,Ficus religiosa,0
2,IMG20231004154748.jpg,Peepla,Ficus religiosa,0
3,IMG20231004154809.jpg,Peepla,Ficus religiosa,0
4,IMG20231004154917.jpg,Peepla,Ficus religiosa,0
...,...,...,...,...
92,IMG20231004172618.jpg,Pipal,Ficus amplissima,19
93,IMG20231004172657.jpg,Pipal,Ficus amplissima,19
94,IMG20231004172723.jpg,Pipal,Ficus amplissima,19
95,IMG20231004172803.jpg,Pipal,Ficus amplissima,19


In [13]:
tmp = dict(zip(range(5), "hello"))
list(tmp.items())[:3]

[(0, 'h'), (1, 'e'), (2, 'l')]

=============================================================================================================================

## Mapping completed and stored in Leaf_Data_Mapped.csv file

-----------------------------------------------------------------------------------------------------------------------------

## Mapping completed of New Species data with Graph Background images

-----------------------------------------------------------------------------------------------------------------------------

[['IMG20240213090615.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091003.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091033.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091053.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091109.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091116.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091130.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091137.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091245.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091302.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091316.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091322.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091334.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091341.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091427.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091438.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091536.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091546.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091555.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091603.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091614.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091624.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091654.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091702.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091923.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092006.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092027.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092107.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092127.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092149.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092217.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092251.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092329.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092336.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092405.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092417.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092547.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092555.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092609.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092614.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092627.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092636.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092644.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092650.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092704.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092713.jpg',
  'Bay Biscayne Creeping Oxeye',
  'Sphagneticola Trilobata',
  23],
 ['IMG20240213092802.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092810.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092824.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092832.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092902.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092910.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092934.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092943.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092954.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213093008.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213093143.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093148.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093159.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093204.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093236.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093244.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093250.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093256.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093314.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093322.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093415.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093438.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093449.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093457.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093505.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093513.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093522.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093527.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093638.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093643.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093655.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093700.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093714.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093720.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093729.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093740.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093759.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093808.jpg',
  "Devil'S Backbone",
  'Pedilanthus Tithymaloides',
  27],
 ['IMG20240213093933.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093939.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093952.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093958.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094008.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094012.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094033.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094042.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094054.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094101.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094115.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094121.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094300.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094310.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094317.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094342.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094408.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094434.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094443.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094527.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094537.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094552.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094600.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094620.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094632.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094656.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094704.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094747.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094752.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094759.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094803.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094816.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094823.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094830.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094837.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094843.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094847.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094938.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213094944.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213094949.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213094953.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213095032.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213095043.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213095058.jpg',
  'Narrow Leaf Dragon Tree',
  'Dracaena Angustifolia',
  32],
 ['IMG20240213095140.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095147.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095157.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095201.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095211.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095216.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095227.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095231.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095243.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095246.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240214085653.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085658.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085706.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085713.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085722.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085730.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085740.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085750.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085803.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085812.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085857.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085904.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085917.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085922.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085954.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090000.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090009.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090016.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090033.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090041.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090124.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090141.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090156.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090203.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090222.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090248.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090300.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090306.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090350.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090356.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090413.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090419.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090428.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090435.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090459.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090506.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090515.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090522.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090537.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090545.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090606.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090615.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090620.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090631.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090638.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090703.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090711.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090745.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090751.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090805.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090824.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090832.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090842.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090849.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090928.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090935.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090943.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090949.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090957.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091003.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091011.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091017.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091037.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091044.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091142.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091149.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091201.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091207.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091214.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091223.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091232.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091239.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091332.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091345.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091454.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091503.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091519.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091530.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091538.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091545.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091552.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091558.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091630.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091634.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091642.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091650.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091715.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091724.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091736.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091744.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091832.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091837.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091846.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091851.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091907.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091913.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091921.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091929.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091940.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091945.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214092139.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092146.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092154.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092200.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092208.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092213.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092226.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092231.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092537.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092546.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092558.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092607.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092627.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092633.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092639.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092645.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092701.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092711.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092749.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092759.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092809.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092820.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092829.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092848.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092908.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092922.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092929.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214093014.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093023.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093033.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093042.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093101.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093106.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093112.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093154.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093200.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093343.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093349.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093359.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093403.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093412.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093417.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093425.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093435.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093539.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093546.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093558.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093604.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093612.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093622.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093630.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093636.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093741.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093747.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093754.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093801.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093823.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093827.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093834.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093840.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093920.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214093925.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214093930.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214093935.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214093943.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214094000.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214094008.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214094012.jpg',
  'Madagascar Rubbervine',
  'Cryptostegia Madagascariensis',
  50],
 ['IMG20240214094058.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094103.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094115.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094121.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094132.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094137.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094144.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094149.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094155.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094201.jpg',
  'Purple False Eranthemum',
  'Pseuderanthemum Carruthersii Var. Atropurpureum',
  51],
 ['IMG20240214094327.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094336.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094346.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094352.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094401.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094407.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094415.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094422.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094600.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094608.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094626.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094633.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094647.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094652.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094706.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094749.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53]]

In [2]:
new_species_mapped_data = [['IMG20240213090615.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091003.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091033.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091053.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091109.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091116.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091130.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091137.jpg', 'Ixora', 'Ixora coccinea', 8],
 ['IMG20240213091245.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091302.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091316.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091322.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091334.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091341.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091427.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091438.jpg', 'Paperflower', 'Bougainvillea Glabra', 20],
 ['IMG20240213091536.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091546.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091555.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091603.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091614.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091624.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091654.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091702.jpg', 'Karonda', 'Carissa Carandas', 21],
 ['IMG20240213091923.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092006.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092027.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092107.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092127.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092149.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092217.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092251.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092329.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092336.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092405.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092417.jpg', 'Pellow Bells', 'Tecoma Stans', 22],
 ['IMG20240213092547.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092555.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092609.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092614.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092627.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092636.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092644.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092650.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092704.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092713.jpg', 'Bay Biscayne Creeping Oxeye', 'Sphagneticola Trilobata', 23],
 ['IMG20240213092802.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092810.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092824.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092832.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092902.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092910.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092934.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092943.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213092954.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213093008.jpg', 'Custard Apple', 'Annona Squamosa', 24],
 ['IMG20240213093143.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093148.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093159.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093204.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093236.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093244.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093250.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093256.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093314.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093322.jpg', 'Lantana', 'Lantana Camara', 25],
 ['IMG20240213093415.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093438.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093449.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093457.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093505.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093513.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093522.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093527.jpg', 'Bili Patra', 'Aegle Marmelos', 26],
 ['IMG20240213093638.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093643.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093655.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093700.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093714.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093720.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093729.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093740.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093759.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093808.jpg', "Devil'S Backbone", 'Pedilanthus Tithymaloides', 27],
 ['IMG20240213093933.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093939.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093952.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213093958.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094008.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094012.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094033.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094042.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094054.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094101.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094115.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094121.jpg', 'Chinese Chaste Tree', 'Vitex Negundo', 28],
 ['IMG20240213094300.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094310.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094317.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094342.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094408.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094434.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094443.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240213094527.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094537.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094552.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094600.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094620.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094632.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094656.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094704.jpg', 'Morning Glory', 'Ipomea Cairica', 30],
 ['IMG20240213094747.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094752.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094759.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094803.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094816.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094823.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094830.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094837.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094843.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094847.jpg', 'Broom Creeper', 'Cocculus Hirsutus', 31],
 ['IMG20240213094938.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213094944.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213094949.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213094953.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213095032.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213095043.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213095058.jpg', 'Narrow Leaf Dragon Tree', 'Dracaena Angustifolia', 32],
 ['IMG20240213095140.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095147.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095157.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095201.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095211.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095216.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095227.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095231.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095243.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240213095246.jpg', 'Oleander', 'Nerium Oleander', 33],
 ['IMG20240214085653.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085658.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085706.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085713.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085722.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085730.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085740.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085750.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085803.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085812.jpg', "Devil'S Horsewhip", 'Achyranthes Aspera', 34],
 ['IMG20240214085857.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085904.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085917.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085922.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214085954.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090000.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090009.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090016.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090033.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090041.jpg', 'Indian Mallow', 'Abutilon Indicon', 35],
 ['IMG20240214090124.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090141.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090156.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090203.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090222.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090248.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090300.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090306.jpg', 'Calico Flower', 'Aristolochia Littoralis', 36],
 ['IMG20240214090350.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090356.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090413.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090419.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090428.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090435.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090459.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090506.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090515.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090522.jpg', 'Morus Indica', 'Tetraploid Mulberry', 37],
 ['IMG20240214090537.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090545.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090606.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090615.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090620.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090631.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090638.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090703.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090711.jpg', "Devil'S Backbone", 'Euphorbia Tithymaloides', 38],
 ['IMG20240214090745.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090751.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090805.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090824.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090832.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090842.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090849.jpg', 'Copper Leaf', 'Acalypha Wilkesiana', 39],
 ['IMG20240214090928.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090935.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090943.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090949.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214090957.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091003.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091011.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091017.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091037.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091044.jpg', 'Kamani', 'Murraya paniculata', 2],
 ['IMG20240214091142.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091149.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091201.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091207.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091214.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091223.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091232.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091239.jpg', '', 'Pelargonium Sp.', 40],
 ['IMG20240214091332.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091345.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091454.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091503.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091519.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091530.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091538.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091545.jpg', 'Henna', 'Lawsonia Inermis', 41],
 ['IMG20240214091552.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091558.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091630.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091634.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091642.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091650.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091715.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091724.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091736.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091744.jpg', 'Christ Thrown', 'Euphorbia Milii', 42],
 ['IMG20240214091832.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091837.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091846.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091851.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091907.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091913.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091921.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091929.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091940.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214091945.jpg', 'Plum Aralia', 'Polyscias Scutellaria', 43],
 ['IMG20240214092139.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092146.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092154.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092200.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092208.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092213.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092226.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092231.jpg', 'Malabar Nut', 'Justicia Adhatoda', 44],
 ['IMG20240214092537.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092546.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092558.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092607.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092627.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092633.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092639.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092645.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092701.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092711.jpg', 'Wood Apple', 'Limonia Acidissima', 45],
 ['IMG20240214092749.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092759.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092809.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092820.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092829.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092848.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092908.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092922.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214092929.jpg', 'Song Of India', 'Dracaena Reflexa', 29],
 ['IMG20240214093014.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093023.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093033.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093042.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093101.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093106.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093112.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093154.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093200.jpg', 'Ming Aralia', 'Polyscias Fruticosa', 46],
 ['IMG20240214093343.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093349.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093359.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093403.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093412.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093417.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093425.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093435.jpg', 'Folha Da Costa', 'Kalanchoe Brasiliensis', 47],
 ['IMG20240214093539.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093546.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093558.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093604.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093612.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093622.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093630.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093636.jpg', 'Indra Pushpa', 'Thunbergia Fragrans', 48],
 ['IMG20240214093741.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093747.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093754.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093801.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093823.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093827.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093834.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093840.jpg', 'Arabian Jasmine', 'Jasminum Sambac', 49],
 ['IMG20240214093920.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214093925.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214093930.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214093935.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214093943.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214094000.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214094008.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214094012.jpg', 'Madagascar Rubbervine', 'Cryptostegia Madagascariensis', 50],
 ['IMG20240214094058.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094103.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094115.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094121.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094132.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094137.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094144.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094149.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094155.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094201.jpg', 'Purple False Eranthemum', 'Pseuderanthemum Carruthersii Var. Atropurpureum', 51],
 ['IMG20240214094327.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094336.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094346.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094352.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094401.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094407.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094415.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094422.jpg', 'Mousehole Tree', 'Myoporum Laetum', 52],
 ['IMG20240214094600.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094608.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094626.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094633.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094647.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094652.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094706.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53],
 ['IMG20240214094749.jpg', "Devil'S Ivy", 'Epipremnun Aureum', 53]]

In [5]:
mld.storeInCSV(new_species_mapped_data)

In [3]:
old_csv_data = pd.read_csv('Git Upload/Leaf_Data_Mapped.csv')
old_csv_data

,Image,Common name,Scientific name,Class number
0,IMG20231004154634.jpg,Peepla,Ficus religiosa,0
1,IMG20231004154651.jpg,Peepla,Ficus religiosa,0
2,IMG20231004154748.jpg,Peepla,Ficus religiosa,0
3,IMG20231004154809.jpg,Peepla,Ficus religiosa,0
4,IMG20231004154917.jpg,Peepla,Ficus religiosa,0
...,...,...,...,...
92,IMG20231004172618.jpg,Pipal,Ficus amplissima,19
93,IMG20231004172657.jpg,Pipal,Ficus amplissima,19
94,IMG20231004172723.jpg,Pipal,Ficus amplissima,19
95,IMG20231004172803.jpg,Pipal,Ficus amplissima,19


In [4]:
all_species_mapped_data = old_csv_data.values.tolist()

for vals in new_species_mapped_data:
    all_species_mapped_data.append(vals)

print(len(old_csv_data.values.tolist()), len(new_species_mapped_data))
len(all_species_mapped_data)

97 332


429

In [6]:
mld.storeInCSV(all_species_mapped_data)